In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)


import numpy as np
from pymt import models
import runmodels_functions as fun

## Coastline Evolution Model + Waves


This example explores how to couple the **Waves** component with the **Coastline Evolution Model** component.

### Links
* [CEM source code](https://github.com/csdms/cem-old): Look at the files that have *deltas* in their name.
* [CEM description on CSDMS](http://csdms.colorado.edu/wiki/Model_help:CEM): Detailed information on the CEM model.

Import the **`CEM`** and **`WAVE`** class, and instantiate it. 

In [ ]:
cem, waves = models.Cem(), models.Waves()

Even though we can't run our waves model yet, we can still get some information about it. *Just don't try to run it.* Some things we can do with our model are get the names of the input variables.

In [ ]:
waves.get_output_var_names()

In [ ]:
cem.get_input_var_names()

Normally we would pass **CEM** a string that represents the name of an input topography file (as we will do during the practical). 

For this example we'll only provide the extent of the simulation and use the defaults.

In [ ]:
args = cem.setup(number_of_rows=100, number_of_cols=200, grid_spacing=200.0)
cem.initialize(*args)
args = waves.setup()
waves.initialize(*args)

We begin with a flat and linear coastline and the bathymetry drops off linearly to the top of the domain.

In [ ]:
grid_id = cem.get_var_grid("sea_water__depth")
spacing = cem.get_grid_spacing(grid_id)
shape = cem.get_grid_shape(grid_id)
z = np.empty(shape)
cem.get_value("sea_water__depth", out=z)
fun.plot_coast(-z,spacing[0],spacing[1])

In [ ]:
waves.set_value(
    "sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter",
    0.3,
)
waves.set_value(
    "sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter",
    0.7,
)
cem.set_value("sea_surface_water_wave__height", 2.0)
cem.set_value("sea_surface_water_wave__period", 7.0)

Allocate memory for the sediment discharge array and set the discharge at the coastal cell to some value.

In [ ]:
qs = np.zeros_like(z)
qs[0, 100] = 1750

Setting wave parameters:

In [ ]:
waves.set_value(
    "sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter",
    0.3,
)
waves.set_value(
    "sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter",
    0.7,
)
cem.set_value("sea_surface_water_wave__height", 2.0)
cem.set_value("sea_surface_water_wave__period", 7.0)

# Running the model

Set the bedload flux and run the model.

In [ ]:
for time in range(3000):
    
    waves.update()
    angle = waves.get_value(
        "sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity"
    )

    cem.set_value(
        "sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity", angle
    )
    cem.set_value("land_surface_water_sediment~bedload__mass_flow_rate", qs)
    cem.update()

cem.get_value("sea_water__depth", out=z)
fun.plot_coast(-z,spacing[0],spacing[1])

Let's add another sediment source with a different flux and update the model.

In [ ]:
qs[0, 150] = 500

for time in range(3750):
    waves.update()
    angle = waves.get_value(
        "sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity"
    )
    cem.set_value(
        "sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity", angle
    )

    cem.set_value("land_surface_water_sediment~bedload__mass_flow_rate", qs)
    cem.update()

cem.get_value("sea_water__depth", out=z)
fun.plot_coast(-z,spacing[0],spacing[1])

Here we shut off the sediment supply completely.

In [ ]:
qs.fill(0.0)

for time in range(4000):
    waves.update()
    angle = waves.get_value(
        "sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity"
    )
    cem.set_value(
        "sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity", angle
    )

    cem.set_value("land_surface_water_sediment~bedload__mass_flow_rate", qs)
    cem.update()

cem.get_value("sea_water__depth", out=z)
fun.plot_coast(-z,spacing[0],spacing[1])